    # Clase - Computación Distribuida

    ## Pyspark Hands-on 
    #### Marcelo Medel Vergara - Diplomado Data Engineer USACH

## Instalación de librerías necesarias

Si utilizas Anaconda para administrar ambientes de desarrollo la mejor vía para asegurar que funcionen correctamente las librerías es instalando directo desde Conda. Adicionalmente se instalan todas las librerías que necesita Spark.
- `conda create -n pyspark-DE`
- `conda activate pyspark-DE`
- `conda install -c conda-forge pyspark python=3.10`

De igual forma se puede instalar a través de PIP, pero sin asegurar funcionamiento correcto ni la instalación de dependencias. Adicionalmente será necesario tener instalado Pandas.
- `conda create -n pyspark-DE python=3.10`
- `conda activate pyspark-DE`
- `pip install pyspark`
- `pip install pandas`.

Para Google Colab:
- `!pip install pyspark`

## SparkSession

SparkSession es una clase en PySpark que existe desde la versión 2.0 (2016) que simplifica la forma de trabajar con Spark, tanto en las configuraciones como en la manipulación de datos estructurados. 


#### Funcionalidades principales de SparkSession:

1. **Configura Spark**: Para profundizar en las configuraciones posibles de Spark, visite https://spark.apache.org/docs/latest/configuration.html.
    - `SparkSession.builder.appName("some name").**config("some config key,value")**.getOrCreate()`
    - `spark.conf.get("some config key")`

2. **Crear DataFrames**: permite leer y escribir (Input/Output) diversas fuentes de datos y crear DataFrames para la manipulación de datos.
    - `spark.createDataFrame(data [, schema])`
    - `spark.read.json("path to some json")`

3. **Ejecutar SQL**: facilita la ejecución de consultas en SQL sobre los DataFrames.
    - `spark.sql("query to some view")`
    
4. **Gestiona contexto de Spark**: facilita la configuración y acceso a diferentes componentes y funcionalidades de Spark
    - `spark.sql.shuffle.partitions`
    - `spark.executor.memory`
    - `spark.catalog.listTables()`
    - `spark.catalog.listColumns("someTable")`
    - `spark.udf.register("someName", someUdf)`

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("hands-on-pyspark")\
        .config("spark.executor.memory","2g")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR") #ALL, OFF, ERROR, DEBUG, INFO, WARN
spark.active()

In [6]:
spark.conf.get("spark.executor.memory")

'2g'

In [7]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'hands-on-pyspark'),
 ('spark.executor.memory', '2g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.app.id', 'local-172

## DataFrame en Spark

Un DataFrame es una estructura de datos bidimensional similar a cualquier tabla en una base de datos estructurada. Algunas de las características que tiene un DataFrame en PySpark son:
- **Distribuido**: Los datos están distribuidos en un clúster de nodos, lo que permite el procesamiento paralelo.

- **Inmutable**: Cada transformación produce un nuevo DataFrame.

- **SQL**: Permite operaciones tipo SQL y ofrece una interfaz similar a Pandas pero a gran escala.

- **Conexiones diversas**: Puede leer datos de múltiples fuentes (ej: JSON, CSV, Parquet, JDBC).

- **Optimización Automática**: Utiliza *Catalyst Optimizer* para optimizar automáticamente las consultas.

### Creación de un DataFrame

In [25]:
from datetime import date
data = [
    {
        "nombre":"Marcelo",
        "nacimiento": date(1987,10,7),
        "mail":"marcelo@gmail.com",
        "x":5689,
        "y":15.26,
        "active":True
    },
    {
        "nombre":"Andrea",
        "nacimiento": date(1990,8,19),
        "mail":"andre@gmail.com",
        "x":4510,
        "y":14.058,
        "active":False
    },
    {
        "nombre":"Juan",
        "nacimiento": date(2000,9,7),
        "mail":"juan@gmail.com",
        "x":3000,
        "y":1.2,
        "active":True
    }
]

df1 = spark.createDataFrame(data)


In [10]:
df1.show()

+------+------+----------+-------+----+------+
|active|género|nacimiento| nombre|   x|     y|
+------+------+----------+-------+----+------+
|  true|     M|1987-10-07|Marcelo|5689| 15.26|
| false|     F|1990-08-19| Andrea|4510|14.058|
|  true|     M|2000-09-07|   Juan|3000|   1.2|
+------+------+----------+-------+----+------+



In [11]:
df1.limit(1).show()

+------+------+----------+-------+----+-----+
|active|género|nacimiento| nombre|   x|    y|
+------+------+----------+-------+----+-----+
|  true|     M|1987-10-07|Marcelo|5689|15.26|
+------+------+----------+-------+----+-----+



In [19]:
from pyspark.sql import Row

df2 = spark.createDataFrame([
    Row(nombre="Marcelo", nacimiento=date(1987,10,7), mail="marcelo.medel.v@gmail.com", x=2356, y=23.56, active=True),
    Row(nombre="Andrea", nacimiento=date(1990,8,19), mail="andrea@gmail.com", x=6352, y=1.56, active=False),
    Row(nombre="Juan", nacimiento=date(2000,9,19), mail="juan@gmail.com", x=5555, y=222.56, active=False)
])

df2.show()

+-------+----------+--------------------+----+------+------+
| nombre|nacimiento|                mail|   x|     y|active|
+-------+----------+--------------------+----+------+------+
|Marcelo|1987-10-07|marcelo.medel.v@g...|2356| 23.56|  true|
| Andrea|1990-08-19|    andrea@gmail.com|6352|  1.56| false|
|   Juan|2000-09-19|      juan@gmail.com|5555|222.56| false|
+-------+----------+--------------------+----+------+------+



In [20]:
df1.printSchema()
df2.printSchema()

root
 |-- active: boolean (nullable = true)
 |-- género: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- nombre: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)

root
 |-- nombre: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)
 |-- active: boolean (nullable = true)



### Data Types

In [27]:
from pyspark.sql.types import (
    StringType, #texto
    IntegerType, # entero
    DateType, # datetime.date
    FloatType, # 2.33
    DecimalType, # DecimalType(10,2)
    BooleanType, #True or False
    ArrayType, # Lista, tupla...
    StructType, # colección de StructField
    StructField, # un campo estructurado
    MapType,
    LongType,
    DoubleType
)

cols = [
    #StructField("nombre", StringType(), False),
    StructField("nombre_completo", StructType([
        StructField("nombre", StringType(), False),
        StructField("apellido", StringType(), False)
    ]) ),
    StructField("nacimiento", DateType(), True),
    StructField("mail", StringType(), True),
    StructField("x", IntegerType(), False),
    StructField("y", FloatType(), False),
    StructField("active", BooleanType(), False),
]

schema = StructType(cols)

df3 = spark.createDataFrame([], schema)

df3.show()
df3.printSchema()

+---------------+----------+----+---+---+------+
|nombre_completo|nacimiento|mail|  x|  y|active|
+---------------+----------+----+---+---+------+
+---------------+----------+----+---+---+------+

root
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



In [32]:
data = [
    {
        "nombre_completo":["Marcelo","Medel"],
        "nacimiento": date(1987,10,7),
        "mail":"marcelo@gmail.com",
        "x":5689,
        "y":15.26,
        "active":True
    },
    {
        "nombre_completo":["Andrea","Vergara"],
        "nacimiento": date(1990,8,19),
        "mail":"andre@gmail.com",
        "x":4510,
        "y":14.058,
        "active":False
    },
    {
        "nombre_completo":["Juan","Bautista"],
        "nacimiento": date(2000,9,7),
        "mail":"juan@gmail.com",
        "x":3000,
        "y":1.2,
        "active":True
    }
]

df = spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+-----------------+----------+-----------------+----+------+------+
|  nombre_completo|nacimiento|             mail|   x|     y|active|
+-----------------+----------+-----------------+----+------+------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|
+-----------------+----------+-----------------+----+------+------+

root
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



## Operaciones básicas sobre un DataFrame

### Select, Columns y Expressions

***Select*** permite seleccionar un set de columnas y también puede ser usado para renombrar o aplicar *expresiones* a las columnas.

***Columns*** en Spark hace referencia a cualquier columna conocida en algun RDBMS, planilla de excel, pandas dataframes, etc. Estas columnas pueden ser seleccionadas, manipuladas o removidas de un DataFrame, por lo que estas operaciones son representadas como ***expressions***

-  `col` y `column` son utilizadas para referencias a columnas en un DataFrame
- `expr` es utilizado para crear expresiones mas complejas, recibe un string con SQL para ejecutar la operación.


In [39]:
from pyspark.sql.functions import col, column, expr

df.select('nombre_completo'
          ,df.nombre_completo
          ,df['nombre_completo']
          ,col("nombre_completo")
          ,column("nombre_completo")
          ,expr("nombre_completo")
          ).show()

+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  nombre_completo|  nombre_completo|  nombre_completo|  nombre_completo|  nombre_completo|  nombre_completo|
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| {Marcelo, Medel}| {Marcelo, Medel}| {Marcelo, Medel}| {Marcelo, Medel}| {Marcelo, Medel}| {Marcelo, Medel}|
|{Andrea, Vergara}|{Andrea, Vergara}|{Andrea, Vergara}|{Andrea, Vergara}|{Andrea, Vergara}|{Andrea, Vergara}|
| {Juan, Bautista}| {Juan, Bautista}| {Juan, Bautista}| {Juan, Bautista}| {Juan, Bautista}| {Juan, Bautista}|
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+



In [40]:
df.select('nombre_completo.*'
        ,'nombre_completo.apellido'
        ,col('nombre_completo.apellido')
        ,expr('nombre_completo.apellido')
        ).show()


+-------+--------+--------+--------+--------+
| nombre|apellido|apellido|apellido|apellido|
+-------+--------+--------+--------+--------+
|Marcelo|   Medel|   Medel|   Medel|   Medel|
| Andrea| Vergara| Vergara| Vergara| Vergara|
|   Juan|Bautista|Bautista|Bautista|Bautista|
+-------+--------+--------+--------+--------+



In [41]:
df.select(
    expr("*"),
    expr("x*1.19 as pago_iva")
).show()

+-----------------+----------+-----------------+----+------+------+--------+
|  nombre_completo|nacimiento|             mail|   x|     y|active|pago_iva|
+-----------------+----------+-----------------+----+------+------+--------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true| 6769.91|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false| 5366.90|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true| 3570.00|
+-----------------+----------+-----------------+----+------+------+--------+



In [42]:
df.selectExpr("*","x*1.19 as pago_iva").show()

+-----------------+----------+-----------------+----+------+------+--------+
|  nombre_completo|nacimiento|             mail|   x|     y|active|pago_iva|
+-----------------+----------+-----------------+----+------+------+--------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true| 6769.91|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false| 5366.90|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true| 3570.00|
+-----------------+----------+-----------------+----+------+------+--------+



In [47]:
df.selectExpr("*","x*1.19 as pago_iva").selectExpr("avg(pago_iva)", "max(pago_iva)", "min(nacimiento)").show()

+-------------+-------------+---------------+
|avg(pago_iva)|max(pago_iva)|min(nacimiento)|
+-------------+-------------+---------------+
|  5235.603333|      6769.91|     1987-10-07|
+-------------+-------------+---------------+



### Literals, withColumn, withColumnRenamed y alias

In [53]:
from pyspark.sql.functions import lit

#df.selectExpr("*","x*1.19 as pago_iva", lit(1).alias("flag")).show()
df.select(expr("*"), lit(1).alias("flag")).show()

df.selectExpr("*", "1 as flag").show()

+-----------------+----------+-----------------+----+------+------+----+
|  nombre_completo|nacimiento|             mail|   x|     y|active|flag|
+-----------------+----------+-----------------+----+------+------+----+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|   1|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|   1|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|   1|
+-----------------+----------+-----------------+----+------+------+----+

+-----------------+----------+-----------------+----+------+------+----+
|  nombre_completo|nacimiento|             mail|   x|     y|active|flag|
+-----------------+----------+-----------------+----+------+------+----+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|   1|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|   1|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|   1|
+-----------------+----------+-----------------+--

In [56]:
df.withColumn("flag",lit(1)).show()

+-----------------+----------+-----------------+----+------+------+----+
|  nombre_completo|nacimiento|             mail|   x|     y|active|flag|
+-----------------+----------+-----------------+----+------+------+----+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|   1|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|   1|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|   1|
+-----------------+----------+-----------------+----+------+------+----+



In [61]:
df.withColumnRenamed("nombre_completo","nombre").select("nombre.nombre").show()
df =  df.withColumnRenamed("nombre_completo","nombre")
df.show()

+-------+
| nombre|
+-------+
|Marcelo|
| Andrea|
|   Juan|
+-------+

+-----------------+----------+-----------------+----+------+------+
|           nombre|nacimiento|             mail|   x|     y|active|
+-----------------+----------+-----------------+----+------+------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|
+-----------------+----------+-----------------+----+------+------+



In [64]:
df = df.withColumnsRenamed({"nombre_completo":"nombre",
                       "x":"pago",
                       "y":"uso"})
df.show()

+-----------------+----------+-----------------+----+------+------+
|           nombre|nacimiento|             mail|pago|   uso|active|
+-----------------+----------+-----------------+----+------+------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|
+-----------------+----------+-----------------+----+------+------+



### Filter, where, drop y cast

In [73]:
#df.filter(df.active).show()
#df.filter("active").show()
#df.filter("active = True").show()
#df.filter("active = 'false'").show()
#df.filter("active = 0").show()

df.filter("pago > 5000").show()
df.filter(df.pago > 5000).show()

+----------------+----------+-----------------+----+-----+------+
|          nombre|nacimiento|             mail|pago|  uso|active|
+----------------+----------+-----------------+----+-----+------+
|{Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689|15.26|  true|
+----------------+----------+-----------------+----+-----+------+

+----------------+----------+-----------------+----+-----+------+
|          nombre|nacimiento|             mail|pago|  uso|active|
+----------------+----------+-----------------+----+-----+------+
|{Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689|15.26|  true|
+----------------+----------+-----------------+----+-----+------+



In [74]:
df.where("pago > 5000").show()
df.where(df.pago > 5000).show()
df.where("active = 1").show()

+----------------+----------+-----------------+----+-----+------+
|          nombre|nacimiento|             mail|pago|  uso|active|
+----------------+----------+-----------------+----+-----+------+
|{Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689|15.26|  true|
+----------------+----------+-----------------+----+-----+------+

+----------------+----------+-----------------+----+-----+------+
|          nombre|nacimiento|             mail|pago|  uso|active|
+----------------+----------+-----------------+----+-----+------+
|{Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689|15.26|  true|
+----------------+----------+-----------------+----+-----+------+

+----------------+----------+-----------------+----+-----+------+
|          nombre|nacimiento|             mail|pago|  uso|active|
+----------------+----------+-----------------+----+-----+------+
|{Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689|15.26|  true|
|{Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|  1.2|  true|
+-------

In [82]:
df.selectExpr("*", "pago*1.19 as pago_iva").where("pago_iva > 5000").show()

+-----------------+----------+-----------------+----+------+------+--------+
|           nombre|nacimiento|             mail|pago|   uso|active|pago_iva|
+-----------------+----------+-----------------+----+------+------+--------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true| 6769.91|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false| 5366.90|
+-----------------+----------+-----------------+----+------+------+--------+



In [88]:
#df.withColumn("pago_str", col("pago").cast("string")).show()
df = df.withColumn("pago_str", col("pago").cast("string"))
df.show()

+-----------------+----------+-----------------+----+------+------+--------+
|           nombre|nacimiento|             mail|pago|   uso|active|pago_str|
+-----------------+----------+-----------------+----+------+------+--------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|    5689|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|    4510|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|    3000|
+-----------------+----------+-----------------+----+------+------+--------+



In [92]:
df = df.drop("pago_str", "active")
df.printSchema()

root
 |-- nombre: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- pago: integer (nullable = false)
 |-- uso: float (nullable = false)



## Pandas DataFrame to Spark DataFrame

![spark-pandas](./files/spark_pandas.png)

In [93]:
from datetime import date

data = [
    {
        "nombre_completo": ["Carla", "Pérez"],
        "nacimiento": date(1985, 2, 14),
        "mail": "carla.p@gmail.com",
        "x": 6012,
        "y": 20.54,
        "active": True
    },
    {
        "nombre_completo": ["Pedro", "González"],
        "nacimiento": date(1992, 11, 3),
        "mail": "pedro.g@gmail.com",
        "x": 4521,
        "y": 9.8,
        "active": False
    },
    {
        "nombre_completo": ["Lucía", "Ramírez"],
        "nacimiento": date(1989, 5, 22),
        "mail": "lucia.r@gmail.com",
        "x": 3790,
        "y": 12.34,
        "active": True
    },
    {
        "nombre_completo": ["Diego", "Martínez"],
        "nacimiento": date(1995, 12, 19),
        "mail": "diego.m@gmail.com",
        "x": 5120,
        "y": 6.87,
        "active": True
    },
    {
        "nombre_completo": ["Ana", "López"],
        "nacimiento": date(1987, 8, 30),
        "mail": "ana.l@gmail.com",
        "x": 4250,
        "y": 10.52,
        "active": False
    },
    {
        "nombre_completo": ["Fernando", "Sánchez"],
        "nacimiento": date(1991, 3, 17),
        "mail": "fernando.s@gmail.com",
        "x": 5789,
        "y": 13.67,
        "active": True
    },
    {
        "nombre_completo": ["Marta", "Rivas"],
        "nacimiento": date(1994, 6, 8),
        "mail": "marta.r@gmail.com",
        "x": 4892,
        "y": 8.9,
        "active": False
    },
    {
        "nombre_completo": ["Luis", "Ortiz"],
        "nacimiento": date(1988, 9, 29),
        "mail": "luis.o@gmail.com",
        "x": 3400,
        "y": 14.6,
        "active": True
    },
    {
        "nombre_completo": ["Elena", "Torres"],
        "nacimiento": date(1993, 4, 15),
        "mail": "elena.t@gmail.com",
        "x": 4603,
        "y": 7.45,
        "active": True
    },
    {
        "nombre_completo": ["Javier", "Fernández"],
        "nacimiento": date(1986, 7, 21),
        "mail": "javier.f@gmail.com",
        "x": 5950,
        "y": 18.7,
        "active": False
    },
    {
        "nombre_completo": ["Patricia", "Suárez"],
        "nacimiento": date(1990, 1, 11),
        "mail": "patricia.s@gmail.com",
        "x": 3290,
        "y": 10.85,
        "active": True
    },
    {
        "nombre_completo": ["Raúl", "Vega"],
        "nacimiento": date(1992, 10, 10),
        "mail": "raul.v@gmail.com",
        "x": 5125,
        "y": 12.2,
        "active": False
    },
    {
        "nombre_completo": ["Sofía", "Gutiérrez"],
        "nacimiento": date(1984, 5, 5),
        "mail": "sofia.g@gmail.com",
        "x": 4701,
        "y": 11.1,
        "active": True
    },
    {
        "nombre_completo": ["David", "Morales"],
        "nacimiento": date(1996, 12, 25),
        "mail": "david.m@gmail.com",
        "x": 4156,
        "y": 9.3,
        "active": True
    },
    {
        "nombre_completo": ["Claudia", "Navarro"],
        "nacimiento": date(1989, 7, 16),
        "mail": "claudia.n@gmail.com",
        "x": 3958,
        "y": 13.9,
        "active": False
    },
    {
        "nombre_completo": ["Pablo", "Reyes"],
        "nacimiento": date(1991, 11, 7),
        "mail": "pablo.r@gmail.com",
        "x": 5012,
        "y": 8.6,
        "active": True
    },
    {
        "nombre_completo": ["Adriana", "Castro"],
        "nacimiento": date(1985, 6, 27),
        "mail": "adriana.c@gmail.com",
        "x": 3700,
        "y": 11.75,
        "active": True
    },
    {
        "nombre_completo": ["Carlos", "Herrera"],
        "nacimiento": date(1993, 2, 12),
        "mail": "carlos.h@gmail.com",
        "x": 4520,
        "y": 7.9,
        "active": False
    },
    {
        "nombre_completo": ["María", "Molina"],
        "nacimiento": date(1995, 9, 1),
        "mail": "maria.m@gmail.com",
        "x": 4398,
        "y": 10.35,
        "active": True
    },
    {
        "nombre_completo": ["Gustavo", "Cruz"],
        "nacimiento": date(1988, 12, 18),
        "mail": "gustavo.c@gmail.com",
        "x": 5803,
        "y": 16.4,
        "active": False
    },
    {
        "nombre_completo": ["Natalia", "Blanco"],
        "nacimiento": date(1990, 3, 22),
        "mail": "natalia.b@gmail.com",
        "x": 4990,
        "y": 9.67,
        "active": True
    }
]

In [95]:
import pandas as pd

pdf = pd.DataFrame(data)
pdf.head()

,nombre_completo,nacimiento,mail,x,y,active
0,"[Carla, Pérez]",1985-02-14,carla.p@gmail.com,6012,20.54,True
1,"[Pedro, González]",1992-11-03,pedro.g@gmail.com,4521,9.80,False
2,"[Lucía, Ramírez]",1989-05-22,lucia.r@gmail.com,3790,12.34,True
3,"[Diego, Martínez]",1995-12-19,diego.m@gmail.com,5120,6.87,True
4,"[Ana, López]",1987-08-30,ana.l@gmail.com,4250,10.52,False


In [96]:
pdf

,nombre_completo,nacimiento,mail,x,y,active
0,"[Carla, Pérez]",1985-02-14,carla.p@gmail.com,6012,20.54,True
1,"[Pedro, González]",1992-11-03,pedro.g@gmail.com,4521,9.80,False
2,"[Lucía, Ramírez]",1989-05-22,lucia.r@gmail.com,3790,12.34,True
3,"[Diego, Martínez]",1995-12-19,diego.m@gmail.com,5120,6.87,True
4,"[Ana, López]",1987-08-30,ana.l@gmail.com,4250,10.52,False
5,"[Fernando, Sánchez]",1991-03-17,fernando.s@gmail.com,5789,13.67,True
6,"[Marta, Rivas]",1994-06-08,marta.r@gmail.com,4892,8.90,False
7,"[Luis, Ortiz]",1988-09-29,luis.o@gmail.com,3400,14.60,True
8,"[Elena, Torres]",1993-04-15,elena.t@gmail.com,4603,7.45,True
9,"[Javier, Fernández]",1986-07-21,javier.f@gmail.com,5950,18.70,False


In [97]:
df = spark.createDataFrame(pdf)
df.show()

+-------------------+----------+--------------------+----+-----+------+
|    nombre_completo|nacimiento|                mail|   x|    y|active|
+-------------------+----------+--------------------+----+-----+------+
|     [Carla, Pérez]|1985-02-14|   carla.p@gmail.com|6012|20.54|  true|
|  [Pedro, González]|1992-11-03|   pedro.g@gmail.com|4521|  9.8| false|
|   [Lucía, Ramírez]|1989-05-22|   lucia.r@gmail.com|3790|12.34|  true|
|  [Diego, Martínez]|1995-12-19|   diego.m@gmail.com|5120| 6.87|  true|
|       [Ana, López]|1987-08-30|     ana.l@gmail.com|4250|10.52| false|
|[Fernando, Sánchez]|1991-03-17|fernando.s@gmail.com|5789|13.67|  true|
|     [Marta, Rivas]|1994-06-08|   marta.r@gmail.com|4892|  8.9| false|
|      [Luis, Ortiz]|1988-09-29|    luis.o@gmail.com|3400| 14.6|  true|
|    [Elena, Torres]|1993-04-15|   elena.t@gmail.com|4603| 7.45|  true|
|[Javier, Fernández]|1986-07-21|  javier.f@gmail.com|5950| 18.7| false|
| [Patricia, Suárez]|1990-01-11|patricia.s@gmail.com|3290|10.85|

In [100]:
pdf.head().T

,0,1,2,3,4
nombre_completo,"[Carla, Pérez]","[Pedro, González]","[Lucía, Ramírez]","[Diego, Martínez]","[Ana, López]"
nacimiento,1985-02-14,1992-11-03,1989-05-22,1995-12-19,1987-08-30
mail,carla.p@gmail.com,pedro.g@gmail.com,lucia.r@gmail.com,diego.m@gmail.com,ana.l@gmail.com
x,6012,4521,3790,5120,4250
y,20.54,9.8,12.34,6.87,10.52
active,True,False,True,True,False


In [98]:
df.printSchema()

root
 |-- nombre_completo: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)
 |-- active: boolean (nullable = true)



In [99]:
df.show(10, vertical=True)

-RECORD 0-------------------------------
 nombre_completo | [Carla, Pérez]       
 nacimiento      | 1985-02-14           
 mail            | carla.p@gmail.com    
 x               | 6012                 
 y               | 20.54                
 active          | true                 
-RECORD 1-------------------------------
 nombre_completo | [Pedro, González]    
 nacimiento      | 1992-11-03           
 mail            | pedro.g@gmail.com    
 x               | 4521                 
 y               | 9.8                  
 active          | false                
-RECORD 2-------------------------------
 nombre_completo | [Lucía, Ramírez]     
 nacimiento      | 1989-05-22           
 mail            | lucia.r@gmail.com    
 x               | 3790                 
 y               | 12.34                
 active          | true                 
-RECORD 3-------------------------------
 nombre_completo | [Diego, Martínez]    
 nacimiento      | 1995-12-19           
 mail           

In [101]:
df = spark.createDataFrame(data)
df.show()
pdf = df.toPandas()
pdf.head()

+------+--------------------+----------+-------------------+----+-----+
|active|                mail|nacimiento|    nombre_completo|   x|    y|
+------+--------------------+----------+-------------------+----+-----+
|  true|   carla.p@gmail.com|1985-02-14|     [Carla, Pérez]|6012|20.54|
| false|   pedro.g@gmail.com|1992-11-03|  [Pedro, González]|4521|  9.8|
|  true|   lucia.r@gmail.com|1989-05-22|   [Lucía, Ramírez]|3790|12.34|
|  true|   diego.m@gmail.com|1995-12-19|  [Diego, Martínez]|5120| 6.87|
| false|     ana.l@gmail.com|1987-08-30|       [Ana, López]|4250|10.52|
|  true|fernando.s@gmail.com|1991-03-17|[Fernando, Sánchez]|5789|13.67|
| false|   marta.r@gmail.com|1994-06-08|     [Marta, Rivas]|4892|  8.9|
|  true|    luis.o@gmail.com|1988-09-29|      [Luis, Ortiz]|3400| 14.6|
|  true|   elena.t@gmail.com|1993-04-15|    [Elena, Torres]|4603| 7.45|
| false|  javier.f@gmail.com|1986-07-21|[Javier, Fernández]|5950| 18.7|
|  true|patricia.s@gmail.com|1990-01-11| [Patricia, Suárez]|3290

,active,mail,nacimiento,nombre_completo,x,y
0,True,carla.p@gmail.com,1985-02-14,"[Carla, Pérez]",6012,20.54
1,False,pedro.g@gmail.com,1992-11-03,"[Pedro, González]",4521,9.80
2,True,lucia.r@gmail.com,1989-05-22,"[Lucía, Ramírez]",3790,12.34
3,True,diego.m@gmail.com,1995-12-19,"[Diego, Martínez]",5120,6.87
4,False,ana.l@gmail.com,1987-08-30,"[Ana, López]",4250,10.52
